In [1]:
import os
import pandas as pd
import numpy as np
import math
from datetime import datetime
import re
import toad
import random
import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
random.seed(202306)

### Data Quality Check

In [3]:
df_wide = pd.read_csv('widetable_cleaned.csv', encoding="utf-8")
print('Shape:', df_wide.shape)
# df_wide.drop(columns='Unnamed: 0', inplace=True)
df_wide.head()
# df_wide.info()

Shape: (25466, 25)


,NATURAL_CUST_ID,target_M2plus,target_owe_fee_rate,FLAG_M2_HISTORY,OWETAG_OWE_FEE_N_RATIO_VL_OLD,OWETAG_OWE_FEE_L12M_RATIO_VL_OLD,OWETAG_OWE_TIME_L12M_MX_OLD,OWETAG_OWE_PERIOD_L12M_MX,COSHQW_ACCT_FEE_N_D3MR,COSHQW_CUST_N_D3M,...,BASIC_CAPITAL,MNG_INFO_MANAGER_N,target_oweRate_binary,REGCAP,IS_RMB,IS_USD,IS_HKD,INDUSTRYCONAME,INDUSTRYCONAME_M2+Prob,INDUSTRYCONAME_oweRateProb
0,122418139,0,0.260940,0,0.163416,0.346650,5,7,-0.109589,0,...,7.079181,2,1,1200.0,1.0,0.0,0.0,其他科技推广服务业,0.087011,0.186205
1,124800149,0,0.009701,0,0.008870,0.010278,5,12,0.112832,0,...,8.698970,2,0,50000.0,1.0,0.0,0.0,其他未列明电气机械及器材制造,0.000000,0.236389
2,140701772,0,159.090909,0,0.000000,0.000000,1,1,1555.555556,0,...,5.698971,2,1,50.0,1.0,0.0,0.0,软件和信息技术服务业,0.129678,0.226528
3,140708146,1,0.166691,1,0.164856,0.164856,3,3,88385.612903,0,...,7.698970,3,0,5000.0,1.0,0.0,0.0,信息技术咨询服务,0.096074,0.250225
4,141101974,0,2.941224,0,0.366565,0.366565,9,12,7.316936,0,...,8.540079,10,1,34680.0,1.0,0.0,0.0,资源与产权交易服务,0.241162,0.441833


In [4]:
df_wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25466 entries, 0 to 25465
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   NATURAL_CUST_ID                        25466 non-null  int64  
 1   target_M2plus                          25466 non-null  int64  
 2   target_owe_fee_rate                    25466 non-null  float64
 3   FLAG_M2_HISTORY                        25466 non-null  int64  
 4   OWETAG_OWE_FEE_N_RATIO_VL_OLD          25466 non-null  float64
 5   OWETAG_OWE_FEE_L12M_RATIO_VL_OLD       25466 non-null  float64
 6   OWETAG_OWE_TIME_L12M_MX_OLD            25466 non-null  int64  
 7   OWETAG_OWE_PERIOD_L12M_MX              25466 non-null  int64  
 8   COSHQW_ACCT_FEE_N_D3MR                 25466 non-null  float64
 9   COSHQW_CUST_N_D3M                      25466 non-null  int64  
 10  FLAG_M1_HISTORY                        25466 non-null  int64  
 11  CO

In [5]:
# 完整性：检查字面缺失值
def check_na(df):
    if np.sum(np.sum(df.isna())) > 0:
        print('>>> 存在缺失值！')
        df_na = pd.DataFrame({'缺失样本数':np.sum(df.isna()), 
                            '缺失值占比':round(np.sum(df.isna())/len(df),6)})
        return df_na
    else:
        print('>>> 无缺失值！')
    
check_na(df_wide)

>>> 无缺失值！


In [6]:
# 唯一性：候选键/主键检查（待确认）
print('>>> 整体自然客户id数：',df_wide['NATURAL_CUST_ID'].nunique())
# df_wide.groupby('month_id')['natural_cust_id'].agg(func=['nunique',lambda x:len(x)])
df_wide['NATURAL_CUST_ID'].agg(func=['nunique',lambda x:len(x)])

>>> 整体自然客户id数： 25466


nunique     25466
<lambda>    25466
Name: NATURAL_CUST_ID, dtype: int64

### TOAD 特征筛选-M2plus

#### 函数及准备阶段

In [7]:
toad.detect(df_wide)
# toad.detect(test)[:5]

,type,size,missing,unique,mean_or_top1,std_or_top2,min_or_top3,1%_or_top4,10%_or_top5,50%_or_bottom5,75%_or_bottom4,90%_or_bottom3,99%_or_bottom2,max_or_bottom1
NATURAL_CUST_ID,int64,25466,0.00%,25466,49920352.870926,32136702.882824,33768758.0,33815306.65,34044419.5,35784410.0,36437667.25,94585937.5,153000612.8,192954355.0
target_M2plus,int64,25466,0.00%,2,0.112189,0.315605,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
target_owe_fee_rate,float64,25466,0.00%,18185,106.921258,9892.050061,-0.084213,0.0,0.0,0.026673,0.147349,0.44671,16.331431,1416666.666667
FLAG_M2_HISTORY,int64,25466,0.00%,2,0.071939,0.258392,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
OWETAG_OWE_FEE_N_RATIO_VL_OLD,float64,25466,0.00%,15108,0.504649,23.045043,-0.065612,0.0,0.0,0.005017,0.067325,0.260574,2.072523,3350.970284
OWETAG_OWE_FEE_L12M_RATIO_VL_OLD,float64,25466,0.00%,19204,0.638646,23.770672,-0.064696,0.0,0.0,0.041221,0.166191,0.452577,3.077886,3350.970284
OWETAG_OWE_TIME_L12M_MX_OLD,int64,25466,0.00%,12,4.617254,3.00981,0.0,0.0,0.0,5.0,7.0,8.0,11.0,11.0
OWETAG_OWE_PERIOD_L12M_MX,int64,25466,0.00%,13,8.519634,4.738074,0.0,0.0,0.0,12.0,12.0,12.0,12.0,12.0
COSHQW_ACCT_FEE_N_D3MR,float64,25466,0.00%,21488,2101.562883,129467.068835,-2.0,-2.0,-0.232826,-0.000687,0.032719,0.460552,109.247594,17000000.0
COSHQW_CUST_N_D3M,int64,25466,0.00%,45,0.127817,7.044413,-32.0,-1.0,0.0,0.0,0.0,0.0,2.0,817.0


In [8]:
to_drop = ['NATURAL_CUST_ID', 'target_owe_fee_rate', 'INDUSTRYCONAME'] # 去掉ID列、另一个目标变量列和过多类别分类变量列（不考虑one-hot）
# IV（Infromation Value），信息价值，用来表示特征对目标预测的贡献程度，即特征的预测能力，IV值越高\特征的预测能力越强，信息贡献程度越高。
# IV 面向的任务必须是有监督的任务；预测目标必须是二分类的。
# 基尼指数（基尼不纯度）：样本集合中一个随机选中的样本被分错的概率。指数越小表示集合中被选中的样本被分错的概率越小，反之集合越不纯。
print("take target_M2plus as target, quality:")
print(toad.quality(df_wide.drop(to_drop+['target_oweRate_binary', 'INDUSTRYCONAME_oweRateProb'], axis=1), 'target_M2plus', iv_only=False))
print("take target_oweRate_binary as target, quality:")
print(toad.quality(df_wide.drop(to_drop+['target_M2plus', 'INDUSTRYCONAME_M2+Prob'], axis=1), 'target_oweRate_binary', iv_only=False))

take target_M2plus as target, quality:
                                             iv      gini   entropy   unique
OWETAG_OWE_PERIOD_L12M_MX              3.985040  0.164667  0.349246     13.0
FLAG_M2_HISTORY                        1.855806  0.132636  0.253199      2.0
FLAG_M1_HISTORY                        0.794073  0.179066  0.309659      2.0
OWETAG_OWE_TIME_L12M_MX_OLD            0.481045  0.198985  0.350785     12.0
FLAG_PREPAY                            0.480477  0.189447  0.327404      2.0
COSHQW_ACCT_FEE_N_D3MR                 0.318278  0.193730  0.351067  21488.0
INDUSTRYCONAME_M2+Prob                 0.242995  0.196518  0.348625    150.0
BASIC_AGE                              0.149022  0.196923  0.351066   1318.0
OWETAG_OWE_FEE_L12M_RATIO_VL_OLD       0.148364  0.198407  0.351067  19204.0
OWETAG_OWE_FEE_N_RATIO_VL_OLD          0.133587  0.198103  0.351067  15108.0
COSHQW_CUST_N_D3M                      0.110474  0.199199  0.351059     45.0
COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD

In [9]:
from sklearn.model_selection import KFold
# 十折交叉
def Split_Sets_10_Fold(total_fold, data):
    
	# total_fold是你设定的几折，我这里之后带实参带10就行，data就是我需要划分的数据
	# train_index, test_index用来存储train和test的index（索引）
    train_index = []
    test_index = []
    kf = KFold(n_splits=total_fold, shuffle=True, random_state=True)
    #这里设置shuffle设置为ture就是打乱顺序在分配
    for train_i, test_i in kf.split(data):
        train_index.append(train_i)
        test_index.append(test_i)
    return train_index, test_index

total_fold = 10
[train_index, test_index] = Split_Sets_10_Fold(total_fold, df_wide)
# train_index 和 test_index 均为长度为10的list，每个元素为对应的行索引的array

In [10]:
def split_set(ind):
    train = df_wide.iloc[train_index[ind]]
    test = df_wide.iloc[test_index[ind]]
    return train, test

In [11]:
from sklearn.linear_model import LogisticRegression
from toad import plot
from toad.metrics import F1, KS, AUC

M2_to_drop = to_drop + ['OWETAG_OWE_PERIOD_L12M_MX', 'FLAG_M2_HISTORY', 'target_oweRate_binary', 'INDUSTRYCONAME_oweRateProb']
rate_bi_to_drop = to_drop + ['OWETAG_OWE_FEE_N_RATIO_VL_OLD', 'OWETAG_OWE_FEE_L12M_RATIO_VL_OLD', 'target_M2plus', 'INDUSTRYCONAME_M2+Prob']

In [12]:
def toad_prog_bin(train, tar:str, another_tar:str)->list:
    if tar == "target_M2plus":
        to_drop = M2_to_drop
    elif tar == "target_oweRate_binary":
        to_drop = rate_bi_to_drop
    else:
        print("error input!")
        return()

    # empyt=0.9: 缺失值大于0.9的变量被删除；iv=0.02: iv值小于0.02的变量被删除；corr=0.7: 两个变量相关性高于0.7时，iv值低的变量被删除
    # return_drop=False: 若为True，function将返回被删去的变量列；exclude=None: 明确不被删去的列名，输入为list格式
    train_selected, dropped = toad.selection.select(train, target=tar, empty=0.9, iv=0.02, corr=0.7,
                                                    return_drop=True, exclude=to_drop)

    print(f"shape of df_wide is: {df_wide.shape}")
    print(f"length of to_drop is: {len(to_drop)}")

    print("features that would be droped according by IV and Corr:")
    print(dropped)
    # print(f"shape of train_selected: {train_selected.shape}")
    # print(f"selected features: {train_selected.columns}")

    # 分箱过程
    c = toad.transform.Combiner()

    # 使用特征筛选后的数据进行训练：稳定的卡方分箱，规定每箱至少有5%数据, 空值将自动被归到最佳箱。支持chi(卡方分箱), dt(决策树分箱), kmean, quantile, step(等步长分箱)
    # min_samples: 每箱至少包含样本量，可以是数字或者占比；n_bins: 箱数，若无法分出这么多箱数，则会分出最多的箱数
    c.fit(train_selected.drop(to_drop, axis=1), y=tar, method='chi', min_samples=0.05) #empty_separate = False

    return(c, train_selected)

#### 得来调M2评分卡的分箱了

In [13]:
train, test = split_set(0)
c, train_selected = toad_prog_bin(train, tar="target_M2plus", another_tar="target_oweRate_binary")

shape of df_wide is: (25466, 25)
length of to_drop is: 7
features that would be droped according by IV and Corr:
{'empty': array([], dtype=float64), 'iv': array(['IS_RMB', 'IS_USD', 'IS_HKD'], dtype=object), 'corr': array(['COSHQW_VHRSK_PROD_FEE_L6M_RATIO_VL',
       'OWETAG_OWE_FEE_N_RATIO_VL_OLD'], dtype=object)}


In [14]:
train_selected.drop(M2_to_drop+['target_M2plus'], axis=1).columns

Index(['OWETAG_OWE_FEE_L12M_RATIO_VL_OLD', 'OWETAG_OWE_TIME_L12M_MX_OLD',
       'COSHQW_ACCT_FEE_N_D3MR', 'COSHQW_CUST_N_D3M', 'FLAG_M1_HISTORY',
       'COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD', 'BASIC_AGE', 'FLAG_PREPAY',
       'BASIC_CAPITAL', 'MNG_INFO_MANAGER_N', 'REGCAP',
       'INDUSTRYCONAME_M2+Prob'],
      dtype='object')

In [ ]:
# 前两个、第6、9、10个没啥好调整分箱的余地（单调性与IV值无法兼顾），第三个（近3个月总出账金额变动比例）肯定得筛掉
# 当前逾期欠费金额与近12个月内总出账金额比率，也不单调；近12个月内逾期欠费次数，次数越多反而坏账率越小？
# 近6个月较高风险类业务出账金额占比，并不是越近0就坏账率越低？注册资本（两列分别是什么意思）、高管人数是不是也是倒U形的坏账率呢？
# 第4个'COSHQW_CUST_N_D3M'近3个月实体客户数变动，可以用0、1切三段单调增————客户变动越多、坏账率越高

for f in train_selected.drop(M2_to_drop+['target_M2plus'], axis=1).columns:
    plot.bin_plot(c.transform(train_selected[[f,'target_M2plus']], labels=True), x=f, target='target_M2plus')
# bar：样本量占比，红线：坏客户占比（坏账率）

In [13]:
# iv值较低，需要重新分箱设置分组
# rule = {'OWETAG_OWE_FEE_L12M_RATIO_VL_OLD': [['O', 'nan'],['F'], ['M']]}
rule_M2 = {
    'OWETAG_OWE_FEE_L12M_RATIO_VL_OLD': [1e-06, 0.000428, 0.150577],
    "OWETAG_OWE_TIME_L12M_MX_OLD": [1, 4, 5, 6, 7, 8],
    "COSHQW_CUST_N_D3M": [0, 1],
    "BASIC_CAPITAL": [0.453318, 7.06695, 8.16376, 8.77298],
    "MNG_INFO_MANAGER_N": [1, 3, 4, 6, 11]
}

# #调整分箱
# c.set_rules(rule_M2)
# # #查看手动分箱稳定性
# plot.bin_plot(c.transform(train_selected[["MNG_INFO_MANAGER_N",'target_M2plus']], labels=True),
#               x="MNG_INFO_MANAGER_N", target='target_M2plus')

### 接着调rate评分卡的分箱

In [19]:
train, test = split_set(0)
c, train_selected = toad_prog_bin(train, tar="target_oweRate_binary", another_tar="target_M2plus")

shape of df_wide is: (25466, 25)
length of to_drop is: 7
features that would be droped according by IV and Corr:
{'empty': array([], dtype=float64), 'iv': array(['FLAG_M2_HISTORY', 'IS_HKD'], dtype=object), 'corr': array(['COSHQW_VHRSK_PROD_FEE_L6M_RATIO_VL', 'FLAG_M1_HISTORY'],
      dtype=object)}


In [20]:
train_selected.drop(rate_bi_to_drop+['target_oweRate_binary'], axis=1).columns

Index(['OWETAG_OWE_TIME_L12M_MX_OLD', 'OWETAG_OWE_PERIOD_L12M_MX',
       'COSHQW_ACCT_FEE_N_D3MR', 'COSHQW_CUST_N_D3M',
       'COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD', 'BASIC_AGE', 'FLAG_PREPAY',
       'BASIC_CAPITAL', 'MNG_INFO_MANAGER_N', 'REGCAP', 'IS_RMB', 'IS_USD',
       'INDUSTRYCONAME_oweRateProb'],
      dtype='object')

In [ ]:
# 第1、2（单增），第3个（'COSHQW_ACCT_FEE_N_D3MR'）很曲折不好办
# 第4个'COSHQW_CUST_N_D3M'近3个月实体客户数变动，可以用0、1切三段单调增————客户变动越多、坏账率越高
# 第5个（'COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD'）大致有点单增的趋势，但前面的箱子不好调
# 第6个（'BASIC_AGE'）有减的趋势，但小有波折，不好调（IV值会低一点）；第7个（单减），有prepay的坏账率低
# 第8\10个，注册资本（两列分别是什么意思）也是倒U形的坏账率？高管人数不是完全的倒U，还是曲折的x
# RMB的坏账率高，USD坏账率低；行业非常稳定的单增

for f in train_selected.drop(rate_bi_to_drop+['target_oweRate_binary'], axis=1).columns:
    plot.bin_plot(c.transform(train_selected[[f,'target_oweRate_binary']], labels=True), x=f, target='target_oweRate_binary')
# plot.bin_plot(c.transform(train[['COSHQW_VHRSK_PROD_FEE_L6M_RATIO_VL','target_oweRate_binary']], labels=True),
#               x='COSHQW_VHRSK_PROD_FEE_L6M_RATIO_VL', target='target_oweRate_binary')

In [14]:
# iv值较低，需要重新分箱设置分组
# rule = {'OWETAG_OWE_FEE_L12M_RATIO_VL_OLD': [['O', 'nan'],['F'], ['M']]}
rule_rate = {
    "OWETAG_OWE_TIME_L12M_MX_OLD": [1, 7, 8, 9],
    "COSHQW_ACCT_FEE_N_D3MR": [-0.168269, -2e-06, 0.368811, 2.325581],
    'COSHQW_CUST_N_D3M': [0, 1],
    "BASIC_AGE": [6.030137, 10.02740, 24.0712329],
    # "BASIC_CAPITAL": [1e-06, 6.670246, 7.714330]
    'MNG_INFO_MANAGER_N': [1, 3, 4, 10],
    "REGCAP": [50.0, 5180.0, 13508.0]
}

# 调整分箱
# c.set_rules(rule_rate)
# # 查看手动分箱稳定性
# plot.bin_plot(c.transform(train_selected[["REGCAP",'target_oweRate_binary']], labels=True),
#               x="REGCAP", target='target_oweRate_binary')

### 完整函数

In [15]:
def toad_prog_M2(train, test, tar:str, another_tar:str)->list:
    c, train_selected = toad_prog_bin(train, tar, another_tar)
    # 调整分箱
    c.set_rules(rule_M2)
    # WOE转化
    # 初始化
    transer = toad.transform.WOETransformer()

    # combiner.transform() & transer.fit_transform() 转化训练数据，并去掉target列
    train_woe = transer.fit_transform(c.transform(train_selected), train_selected[tar], exclude=M2_to_drop+[tar])
    # 根据训练好的transer，转化test/OOT数据
    test_woe = transer.transform(c.transform(test))

    # print("head 5 of train_woe is:")
    # print(train_woe.head(5))
    # M21_train_woe
    # M21_train_woe.shape()

    # 逐步回归第二次筛选特征，将woe转化后的数据做逐步回归

    # estimator: 用于拟合的模型，支持'ols', 'lr', 'lasso', 'ridge'
    # direction: 逐步回归的方向，支持'forward', 'backward', 'both' （推荐）
    # criterion: 评判标准，支持'aic', 'bic', 'ks', 'auc'
    # max_iter: 最大循环次数
    # return_drop: 是否返回被剔除的列名
    final_data = toad.selection.stepwise(train_woe, target=tar, estimator='lr',
                                        direction='both', criterion='aic', exclude=M2_to_drop)
    # 将选出的变量应用于test/OOT数据
    final_test = test_woe[final_data.columns]

    print(f"shape of final_data: {final_data.shape}") # 逐步回归从x个变量中选出了x个
    
    # 最后进入评分卡的变量
    fea = list(final_data.drop(M2_to_drop+[tar],axis=1).columns)
    print(f"final features are: {fea}")

    print("start logistic regression.")
    # 逻辑回归
    lr = LogisticRegression()
    lr.fit(final_data[fea], final_data[tar])

    # 预测训练和测试的OOT
    pred_train = lr.predict_proba(final_data[fea])[:,1]
    pred_test = lr.predict_proba(final_test[fea])[:,1]

    # 模型结果的F1/KS/AUC
    cri = [{"F1": F1(pred_train, final_data[tar]),
        "KS": KS(pred_train, final_data[tar]),
        "AUC": AUC(pred_train, final_data[tar])}]

    print("model diagnosis.")
    # 变量PSI
    PSI_var = toad.metrics.PSI(final_data[fea], final_test[fea])
    # KS bucket 报告
    tr_bucket = toad.metrics.KS_bucket(pred_train, final_data[tar], bucket=10, method='quantile')

    # 将逻辑回归模型转标准评分卡，支持传入逻辑回归参数，进行调参。
    # combiner: 传入训练好的 toad.Combiner 对象；transer: 传入先前训练的 toad.WOETransformer 对象
    # e.g. pdo=60, rate=2, base_odds=20, base_score=750, 实际意义为当比率为1/20，输出基准评分750，当比率为基准比率2倍时，基准分下降60分
    # card: 支持传入专家评分卡
    card = toad.ScoreCard(
        combiner = c, transer = transer,
        class_weight = 'balanced',
        C=0.1, base_score=600, base_odds=10, pdo=50, rate=2
    )

    card.fit(final_data[fea], final_data[tar])
    # print("final card is:")
    final_card = card.export(to_frame = True,)
    # print(final_card)

    return(final_card, fea, cri, PSI_var, tr_bucket)

In [16]:
def toad_prog_rate(train, test, tar:str, another_tar:str)->list:
    c, train_selected = toad_prog_bin(train, tar, another_tar)
    # 调整分箱
    c.set_rules(rule_rate)
    # WOE转化
    # 初始化
    transer = toad.transform.WOETransformer()

    # combiner.transform() & transer.fit_transform() 转化训练数据，并去掉target列
    train_woe = transer.fit_transform(c.transform(train_selected), train_selected[tar], exclude=rate_bi_to_drop+[tar])
    # 根据训练好的transer，转化test/OOT数据
    test_woe = transer.transform(c.transform(test))

    # print("head 5 of train_woe is:")
    # print(train_woe.head(5))
    # M21_train_woe
    # M21_train_woe.shape()

    # 逐步回归第二次筛选特征，将woe转化后的数据做逐步回归

    # estimator: 用于拟合的模型，支持'ols', 'lr', 'lasso', 'ridge'
    # direction: 逐步回归的方向，支持'forward', 'backward', 'both' （推荐）
    # criterion: 评判标准，支持'aic', 'bic', 'ks', 'auc'
    # max_iter: 最大循环次数
    # return_drop: 是否返回被剔除的列名
    final_data = toad.selection.stepwise(train_woe, target=tar, estimator='lr',
                                        direction='both', criterion='aic', exclude=rate_bi_to_drop)
    # 将选出的变量应用于test/OOT数据
    final_test = test_woe[final_data.columns]

    print(f"shape of final_data: {final_data.shape}") # 逐步回归从x个变量中选出了x个
    
    # 最后进入评分卡的变量
    fea = list(final_data.drop(rate_bi_to_drop+[tar],axis=1).columns)
    print(f"final features are: {fea}")

    print("start logistic regression.")
    # 逻辑回归
    lr = LogisticRegression()
    lr.fit(final_data[fea], final_data[tar])

    # 预测训练和测试的OOT
    pred_train = lr.predict_proba(final_data[fea])[:,1]
    pred_test = lr.predict_proba(final_test[fea])[:,1]

    # 模型结果的F1/KS/AUC
    cri = [{"F1": F1(pred_train, final_data[tar]),
        "KS": KS(pred_train, final_data[tar]),
        "AUC": AUC(pred_train, final_data[tar])}]

    print("model diagnosis.")
    # 变量PSI
    PSI_var = toad.metrics.PSI(final_data[fea], final_test[fea])
    # 模型PSI
    # PSI_model = toad.metrics.PSI(pred_train, pred_test)
    # KS bucket 报告
    tr_bucket = toad.metrics.KS_bucket(pred_train, final_data[tar], bucket=10, method='quantile')

    # 将逻辑回归模型转标准评分卡，支持传入逻辑回归参数，进行调参。
    # combiner: 传入训练好的 toad.Combiner 对象；transer: 传入先前训练的 toad.WOETransformer 对象
    # e.g. pdo=60, rate=2, base_odds=20, base_score=750, 实际意义为当比率为1/20，输出基准评分750，当比率为基准比率2倍时，基准分下降60分
    # card: 支持传入专家评分卡
    card = toad.ScoreCard(
        combiner = c, transer = transer,
        class_weight = 'balanced',
        C=0.1, base_score=600, base_odds=10, pdo=50, rate=2
    )

    card.fit(final_data[fea], final_data[tar])
    # print("final card is:")
    final_card = card.export(to_frame = True,)
    # print(final_card)

    return(final_card, fea, cri, PSI_var, tr_bucket)

#### 以M2plus为target

In [17]:
cards = []
feas = []
cris = []
PSI_vars = []
# PSI_models = []
tr_buckets = []
for i in range(10):  # 十折交叉验证
    train, test = split_set(i)
    card, fea, cri, PSI_var, tr_bucket = toad_prog_M2(train, test, tar="target_M2plus", another_tar="target_rate_bi")
    cards.append(card)
    feas.append(fea)
    cris.append(cri)
    PSI_vars.append(PSI_var)
    # PSI_models.append(PSI_model)
    tr_buckets.append(tr_bucket)

shape of df_wide is: (25466, 25)
length of to_drop is: 7
features that would be droped according by IV and Corr:
{'empty': array([], dtype=float64), 'iv': array(['IS_RMB', 'IS_USD', 'IS_HKD'], dtype=object), 'corr': array(['COSHQW_VHRSK_PROD_FEE_L6M_RATIO_VL',
       'OWETAG_OWE_FEE_N_RATIO_VL_OLD'], dtype=object)}
shape of final_data: (22919, 18)
final features are: ['OWETAG_OWE_FEE_L12M_RATIO_VL_OLD', 'OWETAG_OWE_TIME_L12M_MX_OLD', 'COSHQW_CUST_N_D3M', 'FLAG_M1_HISTORY', 'COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD', 'BASIC_AGE', 'FLAG_PREPAY', 'BASIC_CAPITAL', 'MNG_INFO_MANAGER_N', 'INDUSTRYCONAME_M2+Prob']
start logistic regression.
model diagnosis.
shape of df_wide is: (25466, 25)
length of to_drop is: 7
features that would be droped according by IV and Corr:
{'empty': array([], dtype=float64), 'iv': array(['IS_RMB', 'IS_USD', 'IS_HKD'], dtype=object), 'corr': array(['COSHQW_VHRSK_PROD_FEE_L6M_RATIO_VL',
       'OWETAG_OWE_FEE_N_RATIO_VL_OLD'], dtype=object)}
shape of final_data: (22919

In [22]:
import_fea_M2 = ['REGCAP', 'COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD', 'BASIC_CAPITAL', 'INDUSTRYCONAME_M2+Prob',
                 'OWETAG_OWE_TIME_L12M_MX_OLD', 'OWETAG_OWE_FEE_L12M_RATIO_VL_OLD', 'BASIC_AGE', 'FLAG_M2_HISTORY',
                 'OWETAG_OWE_PERIOD_L12M_MX']

In [23]:
for i in range(10):
    cnt = 0
    for fea in import_fea_M2:
        if fea in feas[i]:
            cnt += 1
    print(f"第 {i+1} 个M2评分卡包含{cnt}个重要特征。")

第 1 个M2评分卡包含6个重要特征。
第 2 个M2评分卡包含7个重要特征。
第 3 个M2评分卡包含6个重要特征。
第 4 个M2评分卡包含6个重要特征。
第 5 个M2评分卡包含5个重要特征。
第 6 个M2评分卡包含5个重要特征。
第 7 个M2评分卡包含5个重要特征。
第 8 个M2评分卡包含4个重要特征。
第 9 个M2评分卡包含6个重要特征。
第 10 个M2评分卡包含5个重要特征。


In [25]:
# cards[:3]
# feas
# cris
PSI_vars

[OWETAG_OWE_FEE_L12M_RATIO_VL_OLD         0.000643
 OWETAG_OWE_TIME_L12M_MX_OLD              0.002066
 COSHQW_CUST_N_D3M                        0.000372
 FLAG_M1_HISTORY                          0.000194
 COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD    0.001820
 BASIC_AGE                                0.001223
 FLAG_PREPAY                              0.000055
 BASIC_CAPITAL                            0.001296
 MNG_INFO_MANAGER_N                       0.001626
 INDUSTRYCONAME_M2+Prob                   0.000642
 dtype: float64,
 OWETAG_OWE_FEE_L12M_RATIO_VL_OLD         0.005411
 OWETAG_OWE_TIME_L12M_MX_OLD              0.003095
 COSHQW_CUST_N_D3M                        0.000935
 FLAG_M1_HISTORY                          0.000328
 COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD    0.000365
 BASIC_AGE                                0.000790
 FLAG_PREPAY                              0.000903
 BASIC_CAPITAL                            0.002045
 MNG_INFO_MANAGER_N                       0.002349
 REGCAP       

In [19]:
# 十折内的评估指标一览
cris = [cri[0] for cri in cris]
df_cri = pd.DataFrame(cris)
df_cri.loc['avg'] = df_cri[['F1', 'KS', 'AUC']].mean(axis=0)
df_cri.loc['max'] = df_cri[['F1', 'KS', 'AUC']].max(axis=0)
df_cri.loc['min'] = df_cri[['F1', 'KS', 'AUC']].min(axis=0)
df_cri

,F1,KS,AUC
0,0.199120,0.561861,0.869883
1,0.201805,0.559562,0.869507
2,0.201664,0.592482,0.879530
3,0.203707,0.587297,0.876525
4,0.202447,0.585779,0.871616
5,0.202447,0.552877,0.867023
6,0.201373,0.582211,0.873611
7,0.201020,0.560890,0.870356
8,0.201169,0.560532,0.868865
9,0.201585,0.561235,0.869773


In [20]:
tr_buckets[1]
# 最终选择1

,min,max,bads,goods,total,bad_rate,good_rate,odds,bad_prop,good_prop,...,cum_bad_rate_rev,cum_bads_prop,cum_bads_prop_rev,cum_goods_prop,cum_goods_prop_rev,cum_total_prop,cum_total_prop_rev,ks,lift,cum_lift
0,0.001679,0.012606,11,2280,2291,0.004801,0.995199,0.004825,0.004277,0.112056,...,0.112221,0.004277,1.000000,0.112056,1.000000,0.099961,1.000000,0.107779,0.042785,1.000000
1,0.012615,0.017311,17,2275,2292,0.007417,0.992583,0.007473,0.006610,0.111810,...,0.124152,0.010886,0.995723,0.223866,0.887944,0.199965,0.900039,0.212979,0.066094,1.106311
2,0.017312,0.023534,23,2270,2293,0.010031,0.989969,0.010132,0.008942,0.111564,...,0.138743,0.019829,0.989114,0.335430,0.776134,0.300013,0.800035,0.315601,0.089382,1.236338
3,0.023545,0.032303,58,2234,2292,0.025305,0.974695,0.025962,0.022551,0.109795,...,0.157140,0.042379,0.980171,0.445225,0.664570,0.400017,0.699987,0.402846,0.225496,1.400271
4,0.032306,0.043297,95,2195,2290,0.041485,0.958515,0.043280,0.036936,0.107878,...,0.179114,0.079316,0.957621,0.553104,0.554775,0.499935,0.599983,0.473788,0.369669,1.596081
5,0.043322,0.060289,147,2146,2293,0.064108,0.935892,0.068500,0.057154,0.105470,...,0.206614,0.136470,0.920684,0.658574,0.446896,0.599983,0.500065,0.522104,0.571265,1.841128
6,0.060303,0.087858,201,2079,2280,0.088158,0.911842,0.096681,0.078149,0.102177,...,0.242256,0.214619,0.863530,0.760751,0.341426,0.699463,0.400017,0.546132,0.785572,2.158732
7,0.087887,0.132128,238,2066,2304,0.103299,0.896701,0.115198,0.092535,0.101538,...,0.293264,0.307154,0.785381,0.862289,0.239249,0.799991,0.300537,0.555135,0.920490,2.613262
8,0.132248,0.307096,404,1888,2292,0.176265,0.823735,0.213983,0.157076,0.092790,...,0.388743,0.464230,0.692846,0.955079,0.137711,0.899996,0.200009,0.490849,1.570694,3.464079
9,0.307219,0.980307,1378,914,2292,0.601222,0.398778,1.507659,0.535770,0.044921,...,0.601222,1.000000,0.535770,1.000000,0.044921,1.000000,0.100004,-0.000000,5.357465,5.357465


In [27]:
cards[1]
# cards[1].to_excel("M2+评分卡.xlsx")

,name,value,score
0,OWETAG_OWE_FEE_L12M_RATIO_VL_OLD,[-inf ~ 1e-06),172.66
1,OWETAG_OWE_FEE_L12M_RATIO_VL_OLD,[1e-06 ~ 0.000428),-93.29
2,OWETAG_OWE_FEE_L12M_RATIO_VL_OLD,[0.000428 ~ 0.150577),25.47
3,OWETAG_OWE_FEE_L12M_RATIO_VL_OLD,[0.150577 ~ inf),39.96
4,OWETAG_OWE_TIME_L12M_MX_OLD,[-inf ~ 1),88.91
5,OWETAG_OWE_TIME_L12M_MX_OLD,[1 ~ 4),-3.53
6,OWETAG_OWE_TIME_L12M_MX_OLD,[4 ~ 5),4.60
7,OWETAG_OWE_TIME_L12M_MX_OLD,[5 ~ 6),25.03
8,OWETAG_OWE_TIME_L12M_MX_OLD,[6 ~ 7),57.81
9,OWETAG_OWE_TIME_L12M_MX_OLD,[7 ~ 8),71.92


#### 以 rate_bi 为target

In [21]:
cards_rate = []
feas_rate = []
cris_rate = []
PSI_vars_rate = []
# PSI_models = []
tr_buckets_rate = []
for i in range(10):  # 十折交叉验证
    train, test = split_set(i)
    card, fea, cri, PSI_var, tr_bucket = toad_prog_rate(train, test, tar="target_oweRate_binary", another_tar="target_M2plus")
    cards_rate.append(card)
    feas_rate.append(fea)
    cris_rate.append(cri)
    PSI_vars_rate.append(PSI_var)
    # PSI_models.append(PSI_model)
    tr_buckets_rate.append(tr_bucket)

shape of df_wide is: (25466, 25)
length of to_drop is: 7
features that would be droped according by IV and Corr:
{'empty': array([], dtype=float64), 'iv': array(['FLAG_M2_HISTORY', 'IS_HKD'], dtype=object), 'corr': array(['COSHQW_VHRSK_PROD_FEE_L6M_RATIO_VL', 'FLAG_M1_HISTORY'],
      dtype=object)}
shape of final_data: (22919, 15)
final features are: ['OWETAG_OWE_TIME_L12M_MX_OLD', 'COSHQW_ACCT_FEE_N_D3MR', 'COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD', 'FLAG_PREPAY', 'BASIC_CAPITAL', 'IS_USD', 'INDUSTRYCONAME_oweRateProb']
start logistic regression.
model diagnosis.
shape of df_wide is: (25466, 25)
length of to_drop is: 7
features that would be droped according by IV and Corr:
{'empty': array([], dtype=float64), 'iv': array(['FLAG_M2_HISTORY', 'IS_HKD'], dtype=object), 'corr': array(['COSHQW_VHRSK_PROD_FEE_L6M_RATIO_VL', 'FLAG_M1_HISTORY'],
      dtype=object)}
shape of final_data: (22919, 18)
final features are: ['OWETAG_OWE_TIME_L12M_MX_OLD', 'COSHQW_ACCT_FEE_N_D3MR', 'COSHQW_CUST_N_D3M'

In [99]:
import_fea_rate = ['MNG_INFO_MANAGER_N', 'BASIC_CAPITAL', 'COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD',
                   'OWETAG_OWE_TIME_L12M_MX_OLD', 'COSHQW_ACCT_FEE_N_D3MR', 'INDUSTRYCONAME_oweRateProb',
                   'OWETAG_OWE_FEE_N_RATIO_VL_OLD']

In [100]:
for i in range(10):
    cnt = 0
    for fea in import_fea_rate:
        if fea in feas_rate[i]:
            cnt += 1
    print(f"第 {i+1} 个rate评分卡包含{cnt}个重要特征。")

第 1 个rate评分卡包含5个重要特征。
第 2 个rate评分卡包含6个重要特征。
第 3 个rate评分卡包含6个重要特征。
第 4 个rate评分卡包含6个重要特征。
第 5 个rate评分卡包含6个重要特征。
第 6 个rate评分卡包含6个重要特征。
第 7 个rate评分卡包含6个重要特征。
第 8 个rate评分卡包含5个重要特征。
第 9 个rate评分卡包含6个重要特征。
第 10 个rate评分卡包含6个重要特征。


In [26]:
# cards_rate
# feas_rate
# cris_rate
PSI_vars_rate
# tr_buckets_rate

[OWETAG_OWE_TIME_L12M_MX_OLD              0.000865
 COSHQW_ACCT_FEE_N_D3MR                   0.000930
 COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD    0.002832
 FLAG_PREPAY                              0.000055
 BASIC_CAPITAL                            0.001663
 IS_USD                                   0.000135
 INDUSTRYCONAME_oweRateProb               0.003313
 dtype: float64,
 OWETAG_OWE_TIME_L12M_MX_OLD              0.001312
 COSHQW_ACCT_FEE_N_D3MR                   0.001269
 COSHQW_CUST_N_D3M                        0.000935
 COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD    0.000087
 BASIC_AGE                                0.002935
 FLAG_PREPAY                              0.000903
 BASIC_CAPITAL                            0.001994
 MNG_INFO_MANAGER_N                       0.003018
 IS_USD                                   0.000150
 INDUSTRYCONAME_oweRateProb               0.002324
 dtype: float64,
 OWETAG_OWE_TIME_L12M_MX_OLD              0.001716
 COSHQW_ACCT_FEE_N_D3MR                   0.0013

In [22]:
# 十折内的评估指标一览
cris_rate = [cri[0] for cri in cris_rate]
df_cri_rate = pd.DataFrame(cris_rate)
df_cri_rate.loc['avg'] = df_cri_rate[['F1', 'KS', 'AUC']].mean(axis=0)
df_cri_rate.loc['max'] = df_cri_rate[['F1', 'KS', 'AUC']].max(axis=0)
df_cri_rate.loc['min'] = df_cri_rate[['F1', 'KS', 'AUC']].min(axis=0)
df_cri_rate

,F1,KS,AUC
0,0.277554,0.536301,0.846896
1,0.281171,0.538031,0.848177
2,0.278783,0.538735,0.847257
3,0.277943,0.533618,0.847204
4,0.278589,0.539559,0.848790
5,0.280333,0.543609,0.851669
6,0.277619,0.530603,0.843648
7,0.279806,0.532023,0.846204
8,0.279031,0.536963,0.847327
9,0.278848,0.532490,0.845179


In [23]:
tr_buckets_rate[5]

,min,max,bads,goods,total,bad_rate,good_rate,odds,bad_prop,good_prop,...,cum_bad_rate_rev,cum_bads_prop,cum_bads_prop_rev,cum_goods_prop,cum_goods_prop_rev,cum_total_prop,cum_total_prop_rev,ks,lift,cum_lift
0,0.000052,0.011344,3,2289,2292,0.001309,0.998691,0.001311,0.000803,0.119331,...,0.163052,0.000803,1.000000,0.119331,1.000000,0.100004,1.000000,0.118528,0.008027,1.000000
1,0.011369,0.026827,33,2240,2273,0.014518,0.985482,0.014732,0.008831,0.116776,...,0.181025,0.009633,0.999197,0.236107,0.880669,0.199180,0.899996,0.226473,0.089040,1.110225
2,0.026833,0.042252,64,2243,2307,0.027742,0.972258,0.028533,0.017126,0.116933,...,0.201645,0.026759,0.990367,0.353039,0.763893,0.299839,0.800820,0.326280,0.170139,1.236690
3,0.042253,0.060335,92,2203,2295,0.040087,0.959913,0.041761,0.024619,0.114847,...,0.226647,0.051378,0.973241,0.467887,0.646961,0.399974,0.700161,0.416508,0.245854,1.390023
4,0.060361,0.085273,158,2134,2292,0.068935,0.931065,0.074039,0.042280,0.111250,...,0.257781,0.093658,0.948622,0.579137,0.532113,0.499978,0.600026,0.485479,0.422781,1.580968
5,0.085302,0.119445,224,2068,2292,0.097731,0.902269,0.108317,0.059941,0.107809,...,0.295550,0.153599,0.906342,0.686946,0.420863,0.599983,0.500022,0.533347,0.599385,1.812605
6,0.119450,0.173370,345,1947,2292,0.150524,0.849476,0.177196,0.092320,0.101501,...,0.345004,0.245919,0.846401,0.788448,0.313054,0.699987,0.400017,0.542528,0.923160,2.115910
7,0.173379,0.271521,560,1732,2292,0.244328,0.755672,0.323326,0.149853,0.090293,...,0.409831,0.395772,0.754081,0.878740,0.211552,0.799991,0.300013,0.482968,1.498463,2.513493
8,0.271603,0.457338,837,1455,2292,0.365183,0.634817,0.575258,0.223976,0.075852,...,0.492583,0.619748,0.604228,0.954593,0.121260,0.899996,0.200009,0.334844,2.239667,3.021008
9,0.457351,0.983327,1421,871,2292,0.619983,0.380017,1.631458,0.380252,0.045407,...,0.619983,1.000000,0.380252,1.000000,0.045407,1.000000,0.100004,-0.000000,3.802349,3.802349


In [27]:
cards_rate[5]
# cards_rate[5].to_excel("rate评分卡.xlsx")

,name,value,score
0,OWETAG_OWE_TIME_L12M_MX_OLD,[-inf ~ 1),169.54
1,OWETAG_OWE_TIME_L12M_MX_OLD,[1 ~ 7),80.87
2,OWETAG_OWE_TIME_L12M_MX_OLD,[7 ~ 8),39.04
3,OWETAG_OWE_TIME_L12M_MX_OLD,[8 ~ 9),-22.24
4,OWETAG_OWE_TIME_L12M_MX_OLD,[9 ~ inf),-94.78
5,COSHQW_ACCT_FEE_N_D3MR,[-inf ~ -0.168269),-2.85
6,COSHQW_ACCT_FEE_N_D3MR,[-0.168269 ~ -2e-06),81.57
7,COSHQW_ACCT_FEE_N_D3MR,[-2e-06 ~ 0.368811),55.30
8,COSHQW_ACCT_FEE_N_D3MR,[0.368811 ~ 2.325581),18.97
9,COSHQW_ACCT_FEE_N_D3MR,[2.325581 ~ inf),-64.22


评分卡在 fit 时使用 WOE 转换后的数据来计算最终的分数，分数一旦计算完成，便无需 WOE 值，可以直接使用 原始数据 进行评分。

### 在原表里添加新分数列

In [104]:
M2bar = {fea:cards[1].loc[cards[1].name == fea, 'value'].tolist() for fea in feas[1]}
for item in M2bar.items():
    l = item[1]
    l = [float(bar.split(" ~ ")[0][1:]) for bar in l]
    M2bar[item[0]] = l

ratebar = {fea:cards_rate[5].loc[cards_rate[5].name == fea, 'value'].tolist() for fea in feas_rate[5]}
for item in ratebar.items():
    l = item[1]
    l = [float(bar.split(" ~ ")[0][1:]) for bar in l]
    ratebar[item[0]] = l

M2bar
ratebar

{'OWETAG_OWE_TIME_L12M_MX_OLD': [-inf, 1.0, 7.0, 8.0, 9.0],
 'COSHQW_ACCT_FEE_N_D3MR': [-inf, -0.168269, -2e-06, 0.368811, 2.325581],
 'COSHQW_CUST_N_D3M': [-inf, 0.0, 1.0],
 'COSHQW_HRSK_PROD_FEE_L6M_RATIO_VL_OLD': [-inf,
  0.00089,
  0.082057,
  0.152555,
  0.411148,
  0.952746],
 'BASIC_AGE': [-inf, 6.030137, 10.0274, 24.0712329],
 'FLAG_PREPAY': [-inf, 1.0],
 'BASIC_CAPITAL': [-inf, 6.0000004342942646, 7.714329768129296],
 'MNG_INFO_MANAGER_N': [-inf, 1.0, 3.0, 4.0, 10.0],
 'IS_USD': [-inf, 1.0],
 'INDUSTRYCONAME_oweRateProb': [-inf,
  0.0152674721307884,
  0.075985906977922,
  0.1177830356563834,
  0.1541506798272582,
  0.2006706954621511,
  0.2513144282809061,
  0.4054651081081644]}

In [105]:
def val_ind_M2(name:str, val):
    bar = M2bar[name]
    ind = 0
    for cut in bar[1:]:
        if val < cut:
            break
        else:
            ind += 1
    return(ind)

def val_ind_rate(name:str, val):
    bar = ratebar[name]
    ind = 0
    for cut in bar[1:]:
        if val < cut:
            break
        else:
            ind += 1
    return(ind)

In [106]:
df_output = pd.read_csv('widetable_cleaned.csv', encoding="utf-8")
df_output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25466 entries, 0 to 25465
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   NATURAL_CUST_ID                        25466 non-null  int64  
 1   target_M2plus                          25466 non-null  int64  
 2   target_owe_fee_rate                    25466 non-null  float64
 3   FLAG_M2_HISTORY                        25466 non-null  int64  
 4   OWETAG_OWE_FEE_N_RATIO_VL_OLD          25466 non-null  float64
 5   OWETAG_OWE_FEE_L12M_RATIO_VL_OLD       25466 non-null  float64
 6   OWETAG_OWE_TIME_L12M_MX_OLD            25466 non-null  int64  
 7   OWETAG_OWE_PERIOD_L12M_MX              25466 non-null  int64  
 8   COSHQW_ACCT_FEE_N_D3MR                 25466 non-null  float64
 9   COSHQW_CUST_N_D3M                      25466 non-null  int64  
 10  FLAG_M1_HISTORY                        25466 non-null  int64  
 11  CO

In [107]:
df_output['M2+_final_score'] = 0
df_output['rate_final_score'] = 0

for name in M2bar.keys():
    # df_output["M2+_ind_" + name] = 0
    inds = [val_ind_M2(name, val) for val in df_output[name].tolist()]
    # df_output["M2+_ind_" + name] = inds
    values = cards[1].loc[cards[1].name == name, "value"].tolist()
    scores = cards[1].loc[cards[1].name == name, "score"].tolist()
    df_output["M2+_bar_" + name] = [values[ind] for ind in inds]
    df_output["M2+_score_" + name] = [scores[ind] for ind in inds]
    df_output["M2+_final_score"] = df_output["M2+_final_score"] + df_output["M2+_score_" + name]

for name in ratebar.keys():
    # df_output["rate_ind_" + name] = 0
    inds = [val_ind_rate(name, val) for val in df_output[name].tolist()]
    # df_output["M2+_ind_" + name] = inds
    values = cards_rate[5].loc[cards_rate[5].name == name, "value"].tolist()
    scores = cards_rate[5].loc[cards_rate[5].name == name, "score"].tolist()
    df_output["rate_bar_" + name] = [values[ind] for ind in inds]
    df_output["rate_score_" + name] = [scores[ind] for ind in inds]
    df_output["rate_final_score"] = df_output["rate_final_score"] + df_output["rate_score_" + name]

In [108]:
df_output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25466 entries, 0 to 25465
Data columns (total 90 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   NATURAL_CUST_ID                                   25466 non-null  int64  
 1   target_M2plus                                     25466 non-null  int64  
 2   target_owe_fee_rate                               25466 non-null  float64
 3   FLAG_M2_HISTORY                                   25466 non-null  int64  
 4   OWETAG_OWE_FEE_N_RATIO_VL_OLD                     25466 non-null  float64
 5   OWETAG_OWE_FEE_L12M_RATIO_VL_OLD                  25466 non-null  float64
 6   OWETAG_OWE_TIME_L12M_MX_OLD                       25466 non-null  int64  
 7   OWETAG_OWE_PERIOD_L12M_MX                         25466 non-null  int64  
 8   COSHQW_ACCT_FEE_N_D3MR                            25466 non-null  float64
 9   COSHQW_CUST_N_D3M

In [111]:
df_output.to_excel("df_output.xlsx")